## Import all required libraries and functions 

In [1]:
# Import necessary libraries
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Lambda, Flatten, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3

import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn, retinanet
from torchvision.transforms import functional as F


<b> saving the Start Time

In [2]:
main_start_time=time.time()

## Code for CNN
<b> 1. Setup and Preprocessing

In [3]:

# Define the root folder
root_folder = 'C:/Users/Soumiz/Downloads/AII 501 Project Pneumonia/ChestXRay2017_nrm' # 'C:/Users/Soumiz/Downloads/Dummy root'

# Get the subfolders in the root folder
subfolders = [f for f in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, f))]

# Assign subfolders to train_folder and test_folder based on their names
for folder in subfolders:
    if 'train' in folder.lower():
        train_folder = os.path.join(root_folder, folder)
        print(f"Found Train Folder --->\t {train_folder}")
    elif 'test' in folder.lower():
        test_folder = os.path.join(root_folder, folder)
        print(f"Found Test Folder --->\t {test_folder}")

# Define class labels dynamically from folder names
class_labels = {}
for element in subfolders:
    element_path = os.path.join(root_folder, element)  # Path identified
    for idx, folder in enumerate(os.listdir(element_path)):
        key=f"{element}_{folder}"
        if 'normal' in folder.lower():
            class_labels[key] = 0
        else:
            class_labels[key] = 1
            
print("\n\t", class_labels)


Found Test Folder --->	 C:/Users/Soumiz/Downloads/AII 501 Project Pneumonia/ChestXRay2017_nrm\test_nrm
Found Train Folder --->	 C:/Users/Soumiz/Downloads/AII 501 Project Pneumonia/ChestXRay2017_nrm\train_nrm

	 {'test_nrm_NORMAL_nrm': 0, 'test_nrm_PNEUMONIA_nrm': 1, 'train_nrm_NORMAL_nrm': 0, 'train_nrm_PNEUMONIA_nrm': 1}


## CNN with Adaptive Pooling

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Lambda, Flatten
from tensorflow.keras.models import Model

# CNN with Global Average Pooling (similar to adaptive pooling)
def build_cnn_with_adaptive_pooling(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = GlobalAveragePooling2D()(x)  # Global Average Pooling instead of Adaptive Pooling
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(2, activation='softmax')(x)  # Assuming binary classification
    model = Model(inputs, outputs)
    return model

# Example usage
if __name__ == "__main__":
    input_shape = (224, 224, 3)  # Example input shape
    cnn_model = build_cnn_with_adaptive_pooling(input_shape)
    cnn_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 110,018 (429.76 KB)

 Trainable params: 110,018 (429.76 KB)

 Non-trainable params: 0 (0.00 B)

### Load and Preprocess Images

In [5]:
# # root_folder = 'C:/Users/Soumiz/Downloads/AII 501 Project Pneumonia/ChestXRay2017_nrm'

# Define ImageDataGenerator for training and testing
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create the data generators
train_generator = train_datagen.flow_from_directory(
    os.path.join(root_folder, 'train_nrm'),
    target_size=(224, 224),
    batch_size=8,                                   # Batch size can be changed to bigger 
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    os.path.join(root_folder, 'test_nrm'),
    target_size=(224, 224),
    batch_size=8,                                   # Batch size can be changed to bigger 
    class_mode='categorical')


Found 5232 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


### Get Pre-trained Model

In [6]:
input_shape = (224, 224, 3)
model_name = 'CNN'   
model = cnn_model


### Compile and Train the Model

In [7]:
# Extract the images (X) and labels (y) from the generator
X, y = [], []

# Iterate over the generator and accumulate the data and labels
for i in range(len(train_generator)):
    data, labels = train_generator.__getitem__(i)  # Use __getitem__ to access batches
    X.extend(data)
    y.extend(labels)

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

In [11]:
from sklearn.model_selection import KFold

# Create k-fold
kfold = KFold(n_splits=5)
epochs_range = 4 
optimal_epochs = []


for train_idx, val_idx in kfold.split(X):
    model = build_cnn_with_adaptive_pooling(input_shape)  # Initialize or reset model
    print('...')
    # Split the data
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    print('~~~')
    best_epoch = None
    best_val_loss = float('inf')

    print('***')
    # Compile the model 
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


    for epoch in range(epochs_range):
        print('###')
        # Train model for one epoch
        model.fit(X_train, y_train, epochs=1, verbose=0)
        
        # Evaluate on validation set
        val_loss, _ = model.evaluate(X_val, y_val, verbose=0)  # Extract only the loss value
        
        
        # Track the best epoch based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_epoch = epoch
            
    optimal_epochs.append(best_epoch)

# Calculate the average optimal epoch
average_optimal_epoch = sum(optimal_epochs) // len(optimal_epochs)

# Final training with the optimal epoch count
model = build_cnn_with_adaptive_pooling(input_shape)

# Compile the model 
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=average_optimal_epoch)

...
~~~
***
###
###
###
###
...
~~~
***
###
###
###
###
...
~~~
***
###
###
###
###
...
~~~
***
###
###
###
###
...
~~~
***
###
###
###
###
Epoch 1/2
164/164 ━━━━━━━━━━━━━━━━━━━━ 211s 1s/step - accuracy: 0.7160 - loss: 0.5971
Epoch 2/2
164/164 ━━━━━━━━━━━━━━━━━━━━ 185s 1s/step - accuracy: 0.7480 - loss: 0.4840


### Evaluate the Model

In [12]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")


C:\Users\Soumiz\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


78/78 ━━━━━━━━━━━━━━━━━━━━ 31s 383ms/step - accuracy: 0.6587 - loss: 0.9861
Test accuracy: 0.6362179517745972


In [13]:
## Evaluation time

In [14]:
Total_time=time.time()-main_start_time
print("Total Execusion time=",Total_time)

Total Execusion time= 10584.663871526718
